In [2]:
from lib.utils import generate_mean_ensemble_metrics, aggregate_pred_dataframe
import os
import pandas as pd
csv_files = [file for file in os.listdir('result_cv/esm-33-gearnet-ensemble/fold_0/intermediate') if 'test' in file]
csv_files.sort()
csv_files = [f'result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/{file}' for file in csv_files]


In [3]:
agg = pd.DataFrame()
for csv_file in csv_files:
    df = aggregate_pred_dataframe([csv_file])
    me = generate_mean_ensemble_metrics(df)
    print(f'Processing {csv_file}, aggregated metrics: {me["mcc"]})')
    agg = pd.concat([agg, pd.DataFrame([me])], ignore_index=True)

agg.aggregate(['mean', 'std'])

Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_0_test.csv, aggregated metrics: 0.6398974958360563)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_1_test.csv, aggregated metrics: 0.6538033338370434)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_2_test.csv, aggregated metrics: 0.6300222626655065)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_3_test.csv, aggregated metrics: 0.6419543626052465)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_4_test.csv, aggregated metrics: 0.6531483630657888)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_5_test.csv, aggregated metrics: 0.6392976676685302)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_6_test.csv, aggregated metrics: 0.6407205730131768)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_7_test.csv, aggregated metrics: 0.6202270492219251)
Processing result_cv/esm

,sensitivity,specificity,accuracy,precision,mcc
mean,0.581340,0.989154,0.968032,0.750541,0.643468
std,0.033313,0.003288,0.001828,0.050549,0.013273


In [11]:
def format_mean_std(row, metric):
    mean_val = row[(metric, 'mean')]
    std_val = row[(metric, 'std')]
    return f"{mean_val:.3f} ± {std_val:.3f}"


result_df = pd.read_csv('result_cv/result_cv.csv')
grouped = result_df.groupby('model_key').agg({
    'mcc': ['mean', 'std'],
    'sensitivity': ['mean', 'std'],
    'precision': ['mean', 'std']
})

grouped['mcc_formatted'] = grouped.apply(lambda row: format_mean_std(row, 'mcc'), axis=1)
grouped['sensitivity_formatted'] = grouped.apply(lambda row: format_mean_std(row, 'sensitivity'), axis=1)
grouped['precision_formatted'] = grouped.apply(lambda row: format_mean_std(row, 'precision'), axis=1)

formatted_df = grouped[['mcc_formatted', 'sensitivity_formatted', 'precision_formatted']]
formatted_df.columns = ['mcc', 'sensitivity', 'precision']
formatted_df = formatted_df.sort_values('mcc', ascending=True)

formatted_df

,mcc,sensitivity,precision
model_key,,,
gearnet,0.458 ± 0.024,0.374 ± 0.038,0.619 ± 0.071
bert,0.487 ± 0.013,0.400 ± 0.038,0.649 ± 0.061
bert-gearnet-ensemble,0.521 ± 0.014,0.389 ± 0.027,0.746 ± 0.016
bert-gearnet,0.521 ± 0.028,0.441 ± 0.095,0.680 ± 0.111
esm-33-gearnet,0.636 ± 0.020,0.551 ± 0.069,0.777 ± 0.061
esm-t33,0.655 ± 0.016,0.582 ± 0.063,0.775 ± 0.052
esm-33-gearnet-ensemble,0.666 ± 0.013,0.564 ± 0.025,0.819 ± 0.014
esm-33-gearnet-ensemble-rus,0.674 ± 0.011,0.759 ± 0.006,0.632 ± 0.018
esm-33-gearnet-resiboost,0.679 ± 0.010,0.597 ± 0.011,0.805 ± 0.018
